## ANA - Nuclear NICD Bleaching

### Notes

- There are two goals to this analysis
    1. Quantify NICD-mCherry bleaching under continuous activation
        - The bleaching constant is then used for fitting the import/export model, see `ANA - nuclear nicd measurement.ipynb`
    2. Plot NICD nuclear localization ratio over time during continuous and pulsatile activation
        - Using the fitted bleaching function to (approximately) correct for bleaching

### Prep

In [ ]:
# Imports

import os
import pickle, dill

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from matplotlib import lines

import scipy.stats as stats
from scipy.optimize import minimize

import sys; sys.path.insert(0, '..')
import optonotch.utilities as utils
import optonotch.modeling as md

### Data Preparation

#### Data Loading

In [ ]:
# Find all relevant dirs and files

# Get all file paths
walk = os.walk('../Data/Measurements/HisGFP/bleaching control/bleaching_continuous')

# Create a dict for all samples
path_dict = {}
for w in walk:
    for f in w[-1]:
        if f.endswith('_NICDbleach.tsv'):
            path_dict[w[0]] = f

# Report
for dpath in path_dict:
    print(dpath, path_dict[dpath])

In [ ]:
# Load the data (and crop to 30min)

data_dict = {}
    
for dpath in path_dict:

    sample_num = os.path.split(dpath)[-1]

    with open(os.path.join(dpath, path_dict[dpath]), 'r') as infile:
        header = infile.readline().strip().split('\t')

    data = pd.read_csv(os.path.join(dpath, path_dict[dpath]), sep='\t')
    data = data.iloc[:30, :]

    data_dict['sample_'+str(sample_num)] = data

# Report 
print('Header:', ', '.join(header))
for sample in data_dict:
    print('  --', sample + ':', data_dict[sample].shape)

In [ ]:
# Plot the raw data

fig, ax = plt.subplots(1, 3, figsize=(14,4))
    
for dkey in data_dict:

    plot_data = data_dict[dkey]

    ax[0].plot(plot_data['time_step'], plot_data['nuc_nicd_mean'],
               c='teal', alpha=0.5)
    ax[0].set_xlabel('time [min]')
    ax[0].set_ylabel('NICD nuclear mean [au]')

    ax[1].plot(plot_data['time_step'], plot_data['cyt_nicd_mean'],
               c='teal', alpha=0.5)
    ax[1].set_xlabel('time [min]')
    ax[1].set_ylabel('NICD cytoplasmic mean [au]')

    ax[2].plot(plot_data['time_step'], plot_data['all_nicd_mean'],
               c='teal', alpha=0.5)
    labeled = True

    ax[2].set_xlabel('time [min]')
    ax[2].set_ylabel('NICD overall mean [au]')
    
plt.tight_layout()
plt.show()

#### Background correction

In [ ]:
for dkey in data_dict:

    # Use lowest mean value as simple background
    mean_bg = np.min(data_dict[dkey]['nuc_nicd_mean'].values)

    # Apply the subtraction
    data_dict[dkey]['nuc_nicd_mean'] = data_dict[dkey]['nuc_nicd_mean'] - mean_bg
    data_dict[dkey]['cyt_nicd_mean'] = data_dict[dkey]['cyt_nicd_mean'] - mean_bg

    # Compute the new overall mean
    # See `ANA - nuclear nicd measurement.ipynb` for details.
    nuc_ni = data_dict[dkey]['nuc_nicd_mean']
    cyt_ni = data_dict[dkey]['cyt_nicd_mean']
    nuc_sz = data_dict[dkey]['nuc_area']
    cyt_sz = data_dict[dkey]['cyt_area']

    data_dict[dkey]['all_nicd_mean'] = (nuc_ni*nuc_sz + cyt_ni*cyt_sz) / (nuc_sz + cyt_sz)

In [ ]:
# Plot the prepared data

fig, ax = plt.subplots(1, 3, figsize=(14,4))
    
for dkey in data_dict:

    plot_data = data_dict[dkey]

    ax[0].plot(plot_data['time_step'], plot_data['nuc_nicd_mean'],
               c='teal', alpha=0.5)
    ax[0].set_xlabel('time [min]')
    ax[0].set_ylabel('NICD nuclear mean [au]')

    ax[1].plot(plot_data['time_step'], plot_data['cyt_nicd_mean'],
               c='teal', alpha=0.5)
    ax[1].set_xlabel('time [min]')
    ax[1].set_ylabel('NICD cytoplasmic mean [au]')

    ax[2].plot(plot_data['time_step'], plot_data['all_nicd_mean'],
               c='teal', alpha=0.5)

    ax[2].set_xlabel('time [min]')
    ax[2].set_ylabel('NICD overall mean [au]')
    
plt.tight_layout()
plt.show()

### Fitting the Degradation Function

#### Simple Proportional Degration Model

- One component: $G$ for GFP
- Simple mass-action degradation with a single rate: $b$ for bleaching


- System:

$G \overset{b}{\rightarrow} \text{null}$


- Differential equation:

$\frac{dG}{dt} = - b \cdot G$


- Translates into exponential decay:

$G(t) = G_{0} \cdot e^{-b \cdot t}$

In [ ]:
# Get relevant data (time and concentrations between t=10 and t=20)
# Note: normalization is simple here; just express it relative to the starting point!

times = []
G_all = []
G_nuc = []
G_cyt = []

for dkey in data_dict:
    
    time_mask  = (data_dict[dkey]['time_step'] >= 10) 
    time_mask &= (data_dict[dkey]['time_step'] <= 20)
    
    times.append(data_dict[dkey]['time_step'].values[time_mask])
    
    g_all = data_dict[dkey]['all_nicd_mean'].values[time_mask]
    g_nuc = data_dict[dkey]['nuc_nicd_mean'].values[time_mask]
    g_cyt = data_dict[dkey]['cyt_nicd_mean'].values[time_mask]
    
    G_all.append( g_all / g_all[0] )
    G_nuc.append( g_nuc / g_nuc[0] )
    G_cyt.append( g_cyt / g_cyt[0] )
    
times = np.concatenate(times)
G_all = np.concatenate(G_all)
G_nuc = np.concatenate(G_nuc)
G_cyt = np.concatenate(G_cyt)

# Check
plt.scatter(times, G_all, color='teal', alpha=0.4)
plt.xlabel('time [min]')
plt.ylabel('NICD overall mean [au]')
plt.show()

In [ ]:
# Prepare the equation for fitting

deg_model = lambda b, G0, t : G0 * np.exp(-b*t)

In [ ]:
# Loss function

def degfit_loss(b, t, data):
    
    # Run prediction
    pred_G = deg_model(b, 1.0, t)
    
    # Compute mean square error
    MSE = np.mean((data - pred_G)**2.0)
    
    # Done
    return MSE

In [ ]:
# Run fit with `minimize`

# Initial guess [ki, ko, N0]
b_0 = 0.1

# Run minimization
b_fit = {}
for exp, G in zip(['all', 'nuc', 'cyt'], [G_all, G_nuc, G_cyt]):
    res = minimize(degfit_loss, b_0, args=(times-times[0], G))
    b_fit[exp] = res.x[0]
    print(exp+':', 'b={:.4}'.format(b_fit[exp]))
    
# ->> The values all end up similar, so using a single one is
#     appropriate, at least as a simple approximation
# ->> `nuc` is more reliable than `cyt` because `cyt` is closer 
#     to the backgroud at this time, but `all` seems to fit 
#     everything the best, so this is what will be used.

In [ ]:
# Inspect the fit

plt.scatter(times, G_all, color='teal', alpha=0.4)

pred_t = np.linspace(0, 10, 100)
plt.plot(pred_t + times[0], deg_model(b_fit['all'], 1.0, pred_t),
         color='darkblue', alpha=0.7, lw=2)

plt.xlabel('time [min]')
plt.ylabel('NICD overall mean [au]')
plt.show()

In [ ]:
# Check on the full data

fig, ax = plt.subplots(1, 3, figsize=(14,4))

for sample in data_dict:
    
    # Prep
    plot_data = data_dict[sample]
    
    time_mask = (data_dict[sample]['time_step'] >= 10) 
    time_mask &= (data_dict[sample]['time_step'] <= 20)
    
    pred_t = np.linspace(0, 30, 100)
    
    # Nuc
    ax[0].plot(plot_data['time_step'], plot_data['nuc_nicd_mean'],
               c='teal', alpha=0.5)
    ax[0].set_xlabel('time [min]')
    ax[0].set_ylabel('NICD nuclear mean [au]')
    
    pred_G = deg_model(b_fit['all'], plot_data['nuc_nicd_mean'].values[time_mask][0], pred_t)
    ax[0].plot(pred_t + plot_data['time_step'].values[time_mask][0], pred_G, '-',
               c='darkblue', alpha=0.5)
    
    # Cyt
    ax[1].plot(plot_data['time_step'], plot_data['cyt_nicd_mean'],
               c='teal', alpha=0.5)
    ax[1].set_xlabel('time [min]')
    ax[1].set_ylabel('NICD cytoplasmic mean [au]')
    
    pred_G = deg_model(b_fit['all'], plot_data['cyt_nicd_mean'].values[time_mask][0], pred_t)
    ax[1].plot(pred_t + plot_data['time_step'].values[time_mask][0], pred_G, '-',
               c='darkblue', alpha=0.5)
    
    # All
    ax[2].plot(plot_data['time_step'], plot_data['all_nicd_mean'],
               c='teal', alpha=0.5)
    ax[2].set_xlabel('time [min]')
    ax[2].set_ylabel('NICD overall mean [au]')
    
    pred_G = deg_model(b_fit['all'], plot_data['all_nicd_mean'].values[time_mask][0], pred_t)
    ax[2].plot(pred_t + plot_data['time_step'].values[time_mask][0], pred_G, '-',
               c='darkblue', alpha=0.5)
    
plt.tight_layout()
plt.show()

# ->> Looks good for a single-parameter estimate!

### Bleaching Correction on Data

This works with a very simple 1st order approximation, where the fraction bleached in a given compartment remains there.

The import-export model (see `ANA - nuclear nicd measurements.ipynb`) is a more rigorous treatment.

In [ ]:
# Grab & prep the pulsatile data as well

# Find files
walk = os.walk('../Data/Measurements/HisGFP/bleaching control/bleaching_pulsatile')

path_dict = {}
for w in walk:
    for f in w[-1]:
        if f.endswith('_NICDbleach.tsv'):
            path_dict[w[0]] = f

# Load data
pulse_dict = {}
for dpath in path_dict:
    
    sample_num = os.path.split(dpath)[-1]
    
    data = pd.read_csv(os.path.join(dpath, path_dict[dpath]), sep='\t')
    data = data.iloc[:30, :]
    
    pulse_dict['sample_'+str(sample_num)] = data

# Background subtraction
for dkey in pulse_dict:
    
    mean_bg = np.min([pulse_dict[dkey]['nuc_nicd_mean'].values[0],
                      pulse_dict[dkey]['nuc_nicd_mean'].values[-1]])
    
    pulse_dict[dkey]['nuc_nicd_mean'] = pulse_dict[dkey]['nuc_nicd_mean'] - mean_bg
    pulse_dict[dkey]['cyt_nicd_mean'] = pulse_dict[dkey]['cyt_nicd_mean'] - mean_bg

    nuc_ni = pulse_dict[dkey]['nuc_nicd_mean']
    cyt_ni = pulse_dict[dkey]['cyt_nicd_mean']
    nuc_sz = pulse_dict[dkey]['nuc_area']
    cyt_sz = pulse_dict[dkey]['cyt_area']

    pulse_dict[dkey]['all_nicd_mean'] = (nuc_ni*nuc_sz + cyt_ni*cyt_sz) / (nuc_sz + cyt_sz)

In [ ]:
# Bleaching correction for continuous

for sample in data_dict:
    
    # Subselect data
    corr_data = data_dict[sample]
    
    # Prep the output
    corr_nuc = np.zeros(corr_data['time_step'].size)
    corr_cyt = np.zeros(corr_data['time_step'].size)
    corr_all = np.zeros(corr_data['time_step'].size)
    
    # The first value remains the same
    corr_nuc[0] = corr_data['nuc_nicd_mean'][0]
    corr_cyt[0] = corr_data['cyt_nicd_mean'][0]
    corr_all[0] = corr_data['all_nicd_mean'][0]
    
    # Total amount bleached
    bl_nuc = 0
    bl_cyt = 0
    bl_all = 0
    
    # Run through time course
    for ts in corr_data['time_step'][1:]:
        
        # Accumulate bleached fraction
        bl_nuc += corr_data['nuc_nicd_mean'][ts-1] - deg_model(b_fit['all'], corr_data['nuc_nicd_mean'][ts-1], 1.0)
        bl_cyt += corr_data['cyt_nicd_mean'][ts-1] - deg_model(b_fit['all'], corr_data['cyt_nicd_mean'][ts-1], 1.0)
        
        # Compute beach-corrected signal
        corr_nuc[ts] = corr_data['nuc_nicd_mean'][ts] + bl_nuc
        corr_cyt[ts] = corr_data['cyt_nicd_mean'][ts] + bl_cyt
    
    # Recompute the area-corrected total means
    corr_all = (corr_nuc * corr_data['nuc_area'] + corr_cyt * corr_data['cyt_area']) / (corr_data['nuc_area'] + corr_data['cyt_area'])
    
    # Save the results
    data_dict[sample]['corr_time_step'] = corr_data['time_step']
    data_dict[sample]['corr_nuc_nicd_mean'] = corr_nuc
    data_dict[sample]['corr_cyt_nicd_mean'] = corr_cyt
    data_dict[sample]['corr_all_nicd_mean'] = corr_all

In [ ]:
# Bleaching correction for pulsatile

for sample in pulse_dict:
    
    # Subselect data
    corr_data = pulse_dict[sample]
    
    # Prep the output
    corr_nuc = np.zeros(corr_data['time_step'].size)
    corr_cyt = np.zeros(corr_data['time_step'].size)
    corr_all = np.zeros(corr_data['time_step'].size)
    
    # The first value remains the same
    corr_nuc[0] = corr_data['nuc_nicd_mean'][0]
    corr_cyt[0] = corr_data['cyt_nicd_mean'][0]
    corr_all[0] = corr_data['all_nicd_mean'][0]
    
    # Total amount bleached
    bl_nuc = 0
    bl_cyt = 0
    bl_all = 0
    
    # Run through time course
    for ti, ts in enumerate((corr_data['time_step'][1:]).astype(np.int)):
        
        # Know when to skip dark phase
        was_dark = (ts - corr_data['time_step'][ti]) > 1
        
        # Accumulate bleached fraction
        if not was_dark:
            bl_nuc += corr_data['nuc_nicd_mean'][ti] - deg_model(b_fit['all'], corr_data['nuc_nicd_mean'][ti], 1.0)
            bl_cyt += corr_data['cyt_nicd_mean'][ti] - deg_model(b_fit['all'], corr_data['cyt_nicd_mean'][ti], 1.0)
        
        # Compute beach-corrected signal
        corr_nuc[ti+1] = corr_data['nuc_nicd_mean'][ti+1] + bl_nuc
        corr_cyt[ti+1] = corr_data['cyt_nicd_mean'][ti+1] + bl_cyt
    
    # Recompute the area-corrected total means
    corr_all = (corr_nuc * corr_data['nuc_area'] + corr_cyt * corr_data['cyt_area']) / (corr_data['nuc_area'] + corr_data['cyt_area'])
    
    # Save the results
    pulse_dict[sample]['corr_time_step'] = corr_data['time_step']
    pulse_dict[sample]['corr_nuc_nicd_mean'] = corr_nuc
    pulse_dict[sample]['corr_cyt_nicd_mean'] = corr_cyt
    pulse_dict[sample]['corr_all_nicd_mean'] = corr_all

In [ ]:
# Plot bleaching-corrected nuclear ratio of continuous and pulsatile

fig, ax = plt.subplots(1, 2, figsize=(9.5,3.5))


### Continuous activation

# For summary stats
all_x = []
all_y = []

# Plot each sample
for sample in data_dict:
    
    plot_data = data_dict[sample]
    
    all_x.append(plot_data['corr_time_step'])
    all_y.append(plot_data['corr_nuc_nicd_mean'] / plot_data['corr_all_nicd_mean'])
    
    ax[0].plot(all_x[-1], all_y[-1],
               c='teal', alpha=0.5, lw=0.8,
               path_effects=[pe.Stroke(linewidth=2.0, foreground='w'), pe.Normal()])

# Show running mean
all_x = np.concatenate(all_x)
all_y = np.concatenate(all_y)
mean_x, mean_y = utils.running_mean(all_x, all_y, window=0.0, min_samples=0)
ax[0].plot(mean_x, mean_y,
           color='teal', lw=1.2, label='mean', alpha=1.0,
           path_effects=[pe.Stroke(linewidth=2.2, alpha=0.8, foreground='w'), pe.Normal()])
   
# Show CI95
ci_x, ci_y = utils.running_CI(all_x, all_y, window=0.0, min_samples=0.0, ci_type='single')
ax[0].fill_between(ci_x, ci_y[:,0], ci_y[:,1], 
                   color='teal', alpha=0.1, linewidth=0.0, label='CI95')   

# Add legend
ax[0].legend(loc='lower right', #bbox_to_anchor=(0.0,0.85), 
             prop={'family':'Arial', 'size':11})

# Prep activation indicator
ylim = ax[0].get_ylim()
xlim = ax[0].get_xlim()
line = lines.Line2D([mean_x[0], mean_x[-1]], [1.89, 1.89],
                    lw=5., color='#0F76BB')
line.set_clip_on(False)
ax[0].add_line(line)

# Cosmetics & labels
ax[0].set_title('continuous', fontsize=15, fontname='Arial')
ax[0].set_xlabel('time [min]', fontsize=15, fontname='Arial')
ax[0].set_ylabel('NICD nuclear ratio', fontsize=15, fontname='Arial')
ax[0].tick_params(axis='both', which='major', labelsize=14)
[tick.set_fontname('Arial') for tick in ax[0].get_xticklabels()]
[tick.set_fontname('Arial') for tick in ax[0].get_yticklabels()]


### Pulsatile activation

# For summary stats
all_x = []
all_y = []

# Plot each sample
for sample in pulse_dict:
    
    plot_data = pulse_dict[sample]
    
    all_x.append(plot_data['corr_time_step'])
    all_y.append(plot_data['corr_nuc_nicd_mean'] / plot_data['corr_all_nicd_mean'])
    
    ax[1].plot(all_x[-1][:5], all_y[-1][:5],
               c='orchid', alpha=0.5, ms=10, lw=0.9,
               path_effects=[pe.Stroke(linewidth=2.2, foreground='w'), pe.Normal()])
    ax[1].plot(all_x[-1][4:6], all_y[-1][4:6], '--',
               c='orchid', alpha=0.3, ms=10, lw=0.9,
               path_effects=[pe.Stroke(linewidth=2.2, foreground='w'), pe.Normal()])
    ax[1].plot(all_x[-1][5:10], all_y[-1][5:10],
               c='orchid', alpha=0.5, ms=10, lw=0.9,
               path_effects=[pe.Stroke(linewidth=2.2, foreground='w'), pe.Normal()])
    ax[1].plot(all_x[-1][9:11], all_y[-1][9:11], '--',
               c='orchid', alpha=0.3, ms=10, lw=0.9,
               path_effects=[pe.Stroke(linewidth=2.2, foreground='w'), pe.Normal()])
    ax[1].plot(all_x[-1][10:15], all_y[-1][10:15],
               c='orchid', alpha=0.5, ms=10, lw=0.9,
               path_effects=[pe.Stroke(linewidth=2.2, foreground='w'), pe.Normal()])

# Show running mean
all_x = np.concatenate(all_x)
all_y = np.concatenate(all_y)
mean_x, mean_y = utils.running_mean(all_x, all_y, window=0.0, min_samples=0)
ax[1].plot(mean_x[:5], mean_y[:5],
           color='orchid', lw=1.4, alpha=1.0, label='mean',
           path_effects=[pe.Stroke(linewidth=2.6, foreground='w'), pe.Normal()])
ax[1].plot(mean_x[4:6], mean_y[4:6], '--',
           color='orchid', lw=1.4, alpha=0.7,
           path_effects=[pe.Stroke(linewidth=2.6, foreground='w'), pe.Normal()])
ax[1].plot(mean_x[5:10], mean_y[5:10], alpha=1.0,
           color='orchid', lw=1.4,
           path_effects=[pe.Stroke(linewidth=2.6, foreground='w'), pe.Normal()])
ax[1].plot(mean_x[9:11], mean_y[9:11], '--',
           color='orchid', lw=1.4, alpha=0.7,
           path_effects=[pe.Stroke(linewidth=2.6, foreground='w'), pe.Normal()])
ax[1].plot(mean_x[10:15], mean_y[10:15], alpha=1.0,
           color='orchid', lw=1.4,
           path_effects=[pe.Stroke(linewidth=2.6, foreground='w'), pe.Normal()])

# Show CI95
ci_x, ci_y = utils.running_CI(all_x, all_y, window=0.0, min_samples=0.0, ci_type='single')
ax[1].fill_between(ci_x[:], ci_y[:,0], ci_y[:,1], 
                   color='orchid', alpha=0.1, linewidth=0.0, label='CI95')   

# Add legend
ax[1].legend(loc='upper left', bbox_to_anchor=(0.0,0.97), 
             prop={'family':'Arial', 'size':11})

# Prep activation indicator
aws = [[mean_x[0],  mean_x[4] ],
       [mean_x[5],  mean_x[9]],
       [mean_x[10], mean_x[14]]]
for aw in aws:
    line = lines.Line2D(aw, [1.89, 1.89],
                        lw=5., color='#0F76BB')
    line.set_clip_on(False)
    ax[1].add_line(line)
    
# Cosmetics & labels
ax[1].set_title('pulsatile', fontsize=15, fontname='Arial')
ax[1].set_xlabel('time [min]', fontsize=15, fontname='Arial')
ax[1].set_ylabel('NICD nuclear ratio', fontsize=15, fontname='Arial')
ax[1].tick_params(axis='both', which='major', labelsize=14)
[tick.set_fontname('Arial') for tick in ax[1].get_xticklabels()]
[tick.set_fontname('Arial') for tick in ax[1].get_yticklabels()]


### Finish

ax[1].set_ylim(0, ax[1].get_ylim()[1])
ax[0].set_ylim(0, ax[1].get_ylim()[1])

plt.tight_layout()
plt.savefig(r'../Figures/2_DE.pdf')
plt.show()

In [ ]:
# Get IO-model-based activation functions as a reference

# Continuous
input_continuous_model = md.input_functions.input_continuous_model

# Pulsatile
# Note: Because the timings of acquisition were ever so slightly different
#       in this experiment, the timings need to be adjusted compared to
#       the function used in the motif modeling!
pulse_step_params = {'N0'        :  0.0,
                     'off_step1' :  4.0,  
                     'on_step2'  : 15.0, 
                     'off_step2' : 19.0, 
                     'on_step3'  : 30.0, 
                     'end_step'  : 37.0}
input_pulsatile_model = md.input_functions.input_pulsatile_model

In [ ]:
# Plot bleaching-corrected nuclear ratio vs prediction from import-export model

fig, ax = plt.subplots(1, 1, figsize=(7,5))


### Continuous activation

# For summary stats
all_x = []
all_y = []

# Plot each sample
legend_label_set = False
for sample in data_dict:
    
    plot_data = data_dict[sample]
    
    all_x.append(plot_data['corr_time_step'])
    all_y.append(plot_data['corr_nuc_nicd_mean'] / plot_data['corr_all_nicd_mean'])
    
    ax.plot(all_x[-1], all_y[-1],
            c='teal', alpha=0.3, 
            label='$continuous$ data' if not legend_label_set else '__no_label__')
    legend_label_set = True


### Pulsatile activation

# For summary stats
all_x = []
all_y = []

# Plot each sample
legend_label_set = False
for sample in pulse_dict:
    
    plot_data = pulse_dict[sample]
    
    all_x.append(plot_data['corr_time_step'])
    all_y.append(plot_data['corr_nuc_nicd_mean'] / plot_data['corr_all_nicd_mean'])
    
    ax.plot(all_x[-1][:5], all_y[-1][:5],
            c='orchid', alpha=0.4, ms=10)
    ax.plot(all_x[-1][4:6], all_y[-1][4:6], '--',
            c='orchid', alpha=0.2, ms=10)
    ax.plot(all_x[-1][5:10], all_y[-1][5:10],
            c='orchid', alpha=0.4, ms=10)
    ax.plot(all_x[-1][9:11], all_y[-1][9:11], '--',
            c='orchid', alpha=0.2, ms=10)
    ax.plot(all_x[-1][10:15], all_y[-1][10:15],
            c='orchid', alpha=0.4, ms=10,
            label='$pulsatile$ data' if not legend_label_set else '__no_label__')
    legend_label_set = True


### Import-export model

# Continuous
ax.plot(np.linspace(0, 29, 100), 
        [2*input_continuous_model(t) 
         for t in np.linspace(0, 29, 100)],
        '-', lw=2.5, color='darkcyan', alpha=0.9, label='$continuous$ model')

# Pulsatile
ax.plot(np.linspace(0, 34, 100), 
        [2*input_pulsatile_model(t, N0=0.0, steps=pulse_step_params) 
         for t in np.linspace(0, 34, 100)],
        '-', lw=2.5, color='darkorchid', alpha=0.8, label='$pulsatile$ model')


### Cosmetics 

# Add legend
handles, labels = ax.get_legend_handles_labels()
order = [0,2,1,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
           ncol=2, fontsize=10, labelspacing=0.1, loc="lower center")
   
# labels &c.
ax.set_xlabel('time [min]', fontsize=14)
ax.set_ylabel('NICD nuclear ratio', fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=13)


### Finish

ax.set_ylim(0, ax.get_ylim()[1])

plt.savefig(r'../Figures/EV1_B.pdf')
plt.show()